In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import datetime
from typing import cast
from data import SCRAPED_STOCK_FILE_PATH

LOAD DATA

In [ ]:
MAIN_INDEX = ["Date","Area","Name"]
SECONDARY_INDEX = ["Area","Name"]

YEARS_BACK_TO_CONSIDER = 5

stocks = (
    pd.read_csv(
        SCRAPED_STOCK_FILE_PATH,
        dtype={
            "Date": "period[D]",
            "Open": "float",
            "High": "float",
            "Low": "float",
            "Close": "float",
            "Volume": "int",
            "Area": "category",
            "Name": "category",
        },
        index_col=MAIN_INDEX,
    )
    .sort_index()
    .loc[datetime.date.today() - pd.DateOffset(years=YEARS_BACK_TO_CONSIDER) :]
)

UNIQUE_AREAS = stocks.index.get_level_values("Area").unique()

In [ ]:
stocks

**TODO:**
- standardize stock prices for each stock

INVESTIGATE STOCKS AREA HOMOGENEITY

In [ ]:
def plot_comparison_in_groups(group : pd.DataFrame):
    group.reset_index("Area")["Close"].unstack("Name").plot(
        title=cast(str, group.index.get_level_values("Area")[0]),
        ylabel="Close Price",
        xlabel="Date",
        legend=True,
    )

    plt.show()

stocks.groupby("Area", observed=True).apply(plot_comparison_in_groups)

INVESTIGATE STOCKS AREA AVERAGE

In [ ]:
def plot_average_in_groups(group: pd.DataFrame):
    group.reset_index().groupby("Date")["Close"].mean().plot(
        title=f"Average Close Prices for {cast(str, group.index.get_level_values("Area")[0])}",
        ylabel="Close Price",
        xlabel="Date",
        legend=True,
    )
    plt.show()


stocks.groupby("Area", observed=True).apply(plot_average_in_groups)